In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import popmachine, itertools, patsy, GPy, scipy, patsy, GPclust
from popmachine.normal import MultivariateNormal

In [ ]:
op = np.zeros((50,100))
op[:,:50] = -np.eye(50)
op[:,50:] = np.eye(50)

In [ ]:
machine = popmachine.Machine('sqlite:///../popmachine_local/.popmachine.db')

In [ ]:
# ds = machine.search(plates=[u'20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
#        u'20150715 PQ 8', u'20150702 PQ 6', u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9'], Strain='ura3', **{'mM PQ':[0.0, 0.083, 0.333], 'M NaCl':['4.2', None]})

ds = machine.search(plates=[u'20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
       u'20150715 PQ 8', u'20150702 PQ 6', u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9', u'20150607 PQ 4'], Strain='ura3', **{'mM PQ':[0.0, 0.083], 'M NaCl':['4.2', None]})
ds.trim(10)
ds.log()
ds.poly_scale(2,groupby=['plate','mM PQ'])
ds.filter()

In [ ]:
ds.data.shape

In [ ]:
X = ds.data.index.values[:,None]
Y = ds.data.values.T

In [ ]:
k_underlying = GPy.kern.RBF(input_dim=1, variance=0.1, lengthscale=0.1)
k_corruption = GPy.kern.RBF(input_dim=1, variance=0.01, lengthscale=0.1) + GPy.kern.White(1, variance=0.001)

m = GPclust.MOHGP(X, k_underlying, k_corruption, Y, K=2, prior_Z='DP', alpha=1.0)
m

In [ ]:
m.optimize()

In [ ]:
m.systematic_splits(verbose=False)

In [ ]:
m

In [ ]:
# ds.data = ds.data.iloc[::6,:]

In [ ]:

melt = ds.melt()
# melt = melt.iloc[::10,:]
# ind = np.random.choice(range(melt.shape[0]), 3000, replace=False)
# melt = melt.iloc[ind,:]

plt.scatter(melt.time, melt.od, c=melt['mM PQ'])

melt.shape

In [ ]:
xgp = melt[['time', 'mM PQ', 'plate', 'number']].copy()
xgp['pq'] = xgp['mM PQ'].astype(float)
xgp['plate-number'] = xgp['plate'].astype(str) + "_" + xgp['number'].astype(str)
xgp[['plate']], plate_labels = xgp.plate.factorize()
xgp[['plate-number']], _ = xgp['plate-number'].factorize()

xgp = patsy.dmatrix('scale(time) + scale(pq) + plate + number + 0', xgp)
designInfo = xgp.design_info

ygp = melt['od'].values[:,None]

In [ ]:
xgp = np.array(xgp)

In [ ]:
kmain = GPy.kern.RBF(2, ARD=True, name='main')

# kbatch = GPy.kern.RBF(2, ARD=True, name='batch')
# krep = GPy.kern.RBF(2, ARD=True, name='rep')

# linear kernels
kbatch = GPy.kern.Linear(1, name='batch-linear') * GPy.kern.RBF(2, ARD=True, name='batch')
krep = GPy.kern.Linear(1, name='rep-linear') *  GPy.kern.RBF(2, ARD=True, name='rep')


# kern = GPy.kern.Hierarchical((kmain, kbatch))
kern = GPy.kern.Hierarchical((kmain, kbatch, krep))

kern

In [ ]:
m = GPy.models.GPRegression(xgp, ygp, kern, normalizer=True)
m.randomize()
m

In [ ]:
m.optimize()
m

In [ ]:
m['.*lengthscale']

In [ ]:
xpred = np.zeros((100,4))
xpred[:,0] = np.tile(np.linspace(xgp[:,0].min(), xgp[:,0].max()), 2)
xpred[:50,1] = xgp[:,1].min()
xpred[50:,1] = xgp[:,1].max()
xpred[:,2] = 1000
xpred[:,3] = 1000

mu, cov = m.predict_noiseless(xpred, full_cov=True, kern=m.kern.main)
pred = popmachine.normal.MultivariateNormal(mu, cov)
pred.plot()

In [ ]:
pred.dot(op).plot()

In [ ]:
mnull = GPy.models.GPRegression(xgp, ygp, GPy.kern.RBF(2, ARD=True), normalizer=True)
mnull.randomize()
mnull.optimize()
mnull

In [ ]:
m

In [ ]:
xpred = np.zeros((100,4))
xpred[:,0] = np.tile(np.linspace(xgp[:,0].min(), xgp[:,0].max()), 2)
xpred[:50,1] = xgp[:,1].min()
xpred[50:,1] = xgp[:,1].max()
xpred[:,2] = 1000
xpred[:,3] = 1000

mu, cov = m.predict_noiseless(xpred, full_cov=True, kern=m.kern.main)
pred = popmachine.normal.MultivariateNormal(mu, cov)

nullPred = popmachine.normal.MultivariateNormal(*mnull.predict_noiseless(xpred, full_cov=True))

nullPred.dot(op).plot(xpred[:50,0],color='r', label='M0')
pred.dot(op).plot(xpred[:50,0],label='M3')

plt.legend()

plt.xlabel('time (normalized)', fontsize=12)
plt.ylabel('OD (normalized)', fontsize=12)

plt.savefig('figures/ura3_0.083mM-PQ_m03.pdf', bbox_inches='tight')

In [ ]:
models = {}
for p in [u'20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
       u'20150715 PQ 8', u'20150702 PQ 6', u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9', u'20150607 PQ 4']:

    ds = machine.search(plates=[p], Strain='ura3', **{'mM PQ':[0.0, 0.083], 'M NaCl':['4.2', None]})
    ds.trim(10)
    ds.log()
    ds.poly_scale(2,groupby=['plate','mM PQ'])
    ds.filter()
    ds.data = ds.data.iloc[::6,:]
    
    melt = ds.melt()
    #melt = melt.iloc[::10,:]
    
    xgp = melt[['time', 'mM PQ', 'plate', 'number']].copy()
    #xgp[['plate']] = xgp.plate.factorize()[0]
    xgp['pq'] = xgp['mM PQ'].astype(float)
    xgp['plate-number'] = xgp['plate'].astype(str) + "_" + xgp['number'].astype(str)
    xgp[['plate']], _ = xgp.plate.factorize()
    xgp[['plate-number']], _ = xgp['plate-number'].factorize()
    
    xgp = patsy.build_design_matrices([designInfo], xgp)[0]
    xgp = np.array(xgp)
    xgp = xgp.astype(float)

    ygp = melt['od'].values[:,None]

    kmain = GPy.kern.RBF(2, ARD=True, name='main')

    temp = GPy.models.GPRegression(xgp, ygp, kmain)
    temp.randomize()
    temp.optimize()
    
    print p, temp.log_likelihood()
    
    models[p] = temp
    

In [ ]:
m.X

In [ ]:
plt.figure(figsize=(9,9))

for i, p in enumerate(models.keys()):    
    
    plt.subplot(3, 3, i+1)
    plt.title(p)
    
    temp = models[p]
    
    mu, cov = temp.predict_noiseless(xpred, full_cov=True)
    tempPred = popmachine.normal.MultivariateNormal(mu, cov)
    #pred.dot(op).plot(xpred[:50,0])
    tempPred.dot(op).plot(xpred[:50,0],color='r')
    
    xpred[:,2] = plate_labels.tolist().index(p)
    xpred[:,3] = 1000
    mu, cov = m.predict_noiseless(xpred, full_cov=True, kern=GPy.kern.Hierarchical([m.kern.main.copy(), m.kern.batch.copy()]))    
    tempPred = popmachine.normal.MultivariateNormal(mu, cov)
    #pred.dot(op).plot(xpred[:50,0])
    tempPred.dot(op).plot(xpred[:50,0],color='g')
    
    plt.plot([xpred[:50,0].min(), xpred[:50,0].max()], [0,0], 'k', lw=3)
    
    plt.ylim(-.69,.31)
    
    if i > 5:
        plt.xlabel('time (normalized)', fontsize=12)
        
    if i % 3 == 0:
        plt.ylabel('OD (normalized)', fontsize=12)
    
plt.tight_layout()

plt.savefig('figures/ura3_0.083mM-PQ_individualBatches.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(6,4))

for i, p in enumerate(models.keys()):    
    
    temp = models[p]
    
    mu, cov = temp.predict_noiseless(xpred, full_cov=True)
    tempPred = popmachine.normal.MultivariateNormal(mu, cov)
    
    tempPred.dot(op).plot(xpred[:50,0],color='r')
    
mu, cov = m.predict_noiseless(xpred, full_cov=True)
pred = popmachine.normal.MultivariateNormal(mu, cov)
pred.dot(op).plot(xpred[:50,0])

plt.plot([xpred[:50,0].min(), xpred[:50,0].max()], [0,0], 'k', lw=3)
    
plt.tight_layout()

plt.savefig('figures/ura3_0.083mM-PQ_batchDistribution.pdf', bbox_inches='tight')